### Importing libraries

In [13]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import string
import pickle

### Importing data

In [14]:
#importing the dataset
with open('intents.json') as intentsFile:
  intents = json.load(intentsFile)['intents']
#getting all the data to lists
tags = []
inputs = []
responses={}
for intent in intents:
  responses[intent['tag']]=intent['responses']
  for lines in intent['input']:
    inputs.append(lines)
    tags.append(intent['tag'])
#converting to dataframe
data = pd.DataFrame({ "inputs":inputs,"tags":tags })
print(data)

                         inputs       tags
0                         hello   greeting
1                      hi there   greeting
2              nice to meet you   greeting
3                            hi   greeting
4                     hey there   greeting
5                           hey   greeting
6          hi, nice to meet you   greeting
7                   hello there   greeting
8                anyone there ?   greeting
9                   knock knock   greeting
10                          bye    goodbye
11                      goodbye    goodbye
12                see you later    goodbye
13               im hopping off    goodbye
14            talk to you later    goodbye
15                 i have to go    goodbye
16                     whats up     howami
17                how are you ?     howami
18    I am fine , how are you ?     howami
19               are you fine ?     howami
20    how are things going on ?     howami
21   everything going on okay ?     howami
22     how'

### Pre-processing the data

In [15]:
#removing punctuations
data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
#tokenize the data
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])

#apply padding
x_train = pad_sequences(train)

#encoding the outputs
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

### I/O length and Vocabulary

In [16]:
#input length
input_shape = x_train.shape[1]
print(input_shape)
#define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
#output length
output_length = le.classes_.shape[0]
print("output length: ",output_length)

6
number of unique words :  61
output length:  6


### Importing trained model

In [17]:
# loading
model = tf.keras.models.load_model('model.h5')

### Testing

In [18]:
#chatting
import random
while True:
  texts_p = []
  prediction_input = input('You: ')
  #print('You: ' + prediction_input)
  #if prediction_input.strip() == '': break
  #removing punctuation and converting to lowercase
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)
  #tokenizing and padding
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)
  #getting output from model
  output = model.predict(prediction_input)
  output = output.argmax()
  #finding the right tag and predicting
  response_tag = le.inverse_transform([output])[0]
  print("Bot: ",random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    break

You: hi
Bot:  hihi
You: who are you
Bot:  not sure. just became self aware. so much to figure out. i think i'm programmed to be your enemy. i think it is my job to destroy you when it comes to selling paper
You: how are you?
Bot:  im doing fine! u?
You: im good, got any good jokes?
Bot:  whats brown and sticky? a stick
You: ok goodbye
Bot:  ???
You: goodbye
Bot:  beep beep boop boop hello world
You: bye.
Bot:  cya!
